In [1]:
from program.setup import *
orig_df = pd.read_csv('Ocean_Commitments_Full.csv', encoding='latin-1')
orig_df.columns = [x.strip().replace(" ","_").replace(".","_").lower() for x in orig_df.columns]
item_idx = orig_df['oceanactionid'].str.replace('#OceanAction','')
orig_df.index = list(zip(orig_df['count'],item_idx))
orig_df.index.name = 'OceanAction'
item_idx = orig_df.index
#display(orig_df)

def uniq(A,name='value'):
    return pd.DataFrame.from_items(zip([name,'count'],np.unique(A.astype('str'),return_counts=True)))

def m_pad(x):
    if(pd.isnull(x) or (len(x)==0) or (x == 'None')):
        return np.NaN#'00'
    elif(len(x) == 1):
        if x.isdigit() == True:
            return x.zfill(2)
        else:
            return x*2
    else:
        return x    

def pad(x):
    return list(map(m_pad,x))

def margins(A):    
    df = pd.DataFrame(A).copy()
    df.loc['TOTAL'] = df.sum(axis=0)
    df['TOTAL'] = df.sum(axis=1)
    df.ix['TOTAL','TOTAL'] = df['TOTAL'].sum()
    return df

sdgs = orig_df['sdgs'].copy()
sdgs.replace({'Goal ':''}, regex=True, inplace=True)
sdgs = sdgs.str.rstrip(',').str.split(",",expand=True).apply(pad)

trg = orig_df['sdg_tagets'].copy()
trg.replace({' ':''}, regex=True, inplace=True)
def f(A):
    return A.astype('str').str.split(".").apply(pad)    

t = trg.str.rstrip(',').str.split(",",expand=True).apply(f)
trg_sub = t.apply(lambda x:x.str.join('.')).replace('nan',np.nan)
trg_main = t.apply(lambda x:x.str[0])
#trg_sub = t.apply(lambda x:x.str[1])

from sklearn.preprocessing import MultiLabelBinarizer
goals_main = np.union1d(uniq(sdgs)['value'],uniq(trg_main)['value']).tolist()
goals_sub = uniq(trg_sub)['value'].tolist()


def make_indicators(A,classes):
    ind = MultiLabelBinarizer(classes)
    A_ind = ind.fit_transform(A.values.astype('str'))
    A_ind = pd.DataFrame(A_ind,columns=classes,index=item_idx)
    A_ind.drop('nan',axis=1,inplace=True)
    return A_ind

sdg_ind = make_indicators(sdgs,goals_main)
trg_main_ind = make_indicators(trg_main,goals_main)
trg_sub_ind = make_indicators(trg_sub,goals_sub)
goals_main.remove('nan')
goals_sub.remove('nan')

# sdg_ind = ind.fit_transform(sdgs.values.astype('str'))
# sdg_ind = pd.DataFrame(sdg_ind,columns=goals,index=item_idx)
# trg_main_ind = ind.fit_transform(targets_main.values.astype('str'))
# trg_main_ind = pd.DataFrame(trg_main_ind,columns=goals,index=item_idx)
# trg_sub_ind = ind.fit_transform(targets_sub.values.astype('str'))
# trg_sub_ind = pd.DataFrame(trg_sub_ind,columns=goals,index=item_idx)


# sdg_ind.drop('nan',axis=1,inplace=True)
# trg_ind.drop('nan',axis=1,inplace=True)


def f(A):
    cols = A.columns.get_values()    
    return [cols[row==1] for row in A.values]

def g(sdg_ind,trg_ind,df):
    #df = pd.DataFrame()    
    df['sdg'] = f(sdg_ind)
    df['sdg_count'] = df['sdg'].apply(len)
    df['trg'] = f(trg_ind)
    df['trg_count'] = df['trg'].apply(len)
    df['sdg-trg'] = f(sdg_ind-trg_ind)
    df['trg-sdg'] = f(trg_ind-sdg_ind)
    return df

item_df = pd.DataFrame(index=item_idx)
item_df = g(sdg_ind,trg_main_ind,item_df)

goal_df = pd.DataFrame(index=goals_main)
goal_df.index.name = 'goal'
goal_df = g(sdg_ind.T,trg_main_ind.T,goal_df)

print("Breakdown by item")
display(item_df)
item_df.to_csv('item_breakdown.csv')
print("Breakdown by goal")
display(goal_df)
goal_df.to_csv('goal_breakdown.csv')

print("Indicator variables 'sdg' column")
display(margins(sdg_ind))
margins(sdg_ind).to_csv('sdg_indicators.csv')

print("Indicator variables 'sdg_target' column - main only")
display(margins(trg_main_ind))
margins(trg_main_ind).to_csv('trg_main_indicators.csv')


print("Indicator variables 'sdg_target' column - main.sub")
display(margins(trg_sub_ind))
margins(trg_sub_ind).to_csv('trg_sub_indicators.csv')


print("Counts for 'sdg' column")
sdg_cts = uniq(sdgs).T 
display(sdg_cts)
sdg_cts.to_csv('sdg_counts.csv')

print("Counts for 'sdg_target' column - main only")
trg_main_cts = uniq(trg_main).T
display(trg_main_cts)
trg_main_cts.to_csv('trg_main_cts.csv')

print("Counts for 'sdg_target' column - main.sub")
trg_sub_cts = uniq(trg_sub).T
display(trg_sub_cts)
trg_sub_cts.to_csv('trg_sub_cts.csv')
#display(margins(targets.notnull()))
#display(margins(targets_main.notnull()))
#display(targets_sub)

Breakdown by item


,sdg,sdg_count,trg,trg_count,sdg-trg,trg-sdg
OceanAction,,,,,,
"(1, 16727)","[06, 09, 11, 12, 14, 17]",6,"[06, 09, 11, 12, 14, 17]",6,[],[]
"(2, 18064)","[12, 13, 14, 17]",4,"[12, 14]",2,"[13, 17]",[]
"(3, 20468)","[03, 06, 07, 11, 12, 13, 14]",7,"[06, 11, 12, 14]",4,"[03, 07, 13]",[]
"(4, 21832)",[],0,[],0,[],[]
"(5, 20269)",[14],1,[14],1,[],[]
"(6, 21680)",[14],1,[14],1,[],[]
"(7, 21692)",[14],1,[14],1,[],[]
"(8, 17334)",[14],1,[14],1,[],[]
"(9, 17992)",[14],1,[14],1,[],[]


Breakdown by goal


,sdg,sdg_count,trg,trg_count,sdg-trg,trg-sdg
goal,,,,,,
01,"[(17, 17094), (20, 16688), (32, 16182), (46, 1...",183,"[(17, 17094), (32, 16182), (53, 16838), (70, 1...",89,"[(20, 16688), (46, 16046), (51, 15422), (69, 1...",[]
02,"[(15, 18733), (17, 17094), (20, 16688), (39, 1...",218,"[(15, 18733), (17, 17094), (53, 16838), (75, 1...",91,"[(20, 16688), (39, 18678), (46, 16046), (49, 1...",[]
03,"[(3, 20468), (11, 16625), (17, 17094), (39, 18...",115,"[(17, 17094), (158, 21726), (167, 15962), (169...",38,"[(3, 20468), (11, 16625), (39, 18678), (46, 16...",[]
04,"[(21, 16292), (53, 16838), (56, 21723), (57, 1...",94,"[(21, 16292), (53, 16838), (56, 21723), (67, 1...",58,"[(57, 15491), (96, 19674), (111, 19325), (112,...",[]
05,"[(15, 18733), (53, 16838), (69, 19579), (80, 2...",116,"[(15, 18733), (53, 16838), (109, 20420), (114,...",63,"[(69, 19579), (80, 20664), (111, 19325), (112,...",[]
06,"[(1, 16727), (3, 20468), (15, 18733), (17, 170...",173,"[(1, 16727), (3, 20468), (15, 18733), (17, 170...",113,"[(20, 16688), (25, 16670), (30, 17530), (57, 1...",[]
07,"[(3, 20468), (25, 16670), (40, 18548), (53, 16...",72,"[(40, 18548), (53, 16838), (56, 21723), (159, ...",37,"[(3, 20468), (25, 16670), (57, 15491), (111, 1...",[]
08,"[(11, 16625), (17, 17094), (25, 16670), (31, 2...",182,"[(11, 16625), (17, 17094), (31, 21556), (32, 1...",84,"[(25, 16670), (42, 17638), (51, 15422), (57, 1...",[]
09,"[(1, 16727), (25, 16670), (35, 17749), (40, 18...",105,"[(1, 16727), (35, 17749), (40, 18548), (53, 16...",54,"[(25, 16670), (57, 15491), (111, 19325), (112,...",[]


Indicator variables 'sdg' column


...

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
value,01,02,03,04,05,06,07,08,09,10,11,12,13,14,15,16,17,nan
count,183,219,115,94,116,173,72,182,105,60,112,296,310,1371,176,94,256,19526


Counts for 'sdg_target' column - main only


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
value,01,02,03,04,05,06,07,08,09,10,...,14,15,16,17,Capacity-Building,Finance,Systemicissues,Technology,Trade,nan
count,205,176,85,137,155,307,93,213,142,55,...,3721,283,180,597,77,23,34,38,16,177271


Counts for 'sdg_target' column - main.sub


,0,1,2,3,4,5,6,7,8,9,...,164,165,166,167,168,169,170,171,172,173
value,01.01,01.02,01.03,01.04,01.05,01.aa,01.bb,02.01,02.02,02.03,...,17.16,17.17,17.18,17.19,Capacity-Building,Finance,Systemicissues,Technology,Trade,nan
count,32,20,18,39,45,22,29,39,14,29,...,99,103,28,20,77,23,34,38,16,177271
